In [7]:
import os
from enum import Enum
import re

In [8]:
# User Input

addons_paths = ['../odoo/addons', '../enterprise']

In [9]:
# Constants

FOLDERS = ['models', 'report', 'wizard', 'wizards']
# FIELD_TYPES = ['Char', 'Text', 'integer', 'float', 'boolean', 'date', 'datetime', 'binary', 'selection', 'many2one', 'one2many', 'many2many']
FIELD_TYPES = ['One2many', 'Many2one', 'Many2many']

In [10]:
# Field Types Enum Data Structure

class FieldTypes(Enum):
    One2many = 'One2many'
    Many2one = 'Many2one'
    Many2many = 'Many2many'

    def __str__(self):
        return self.value

    @staticmethod
    def get_field_type(field_type: str):
        for ft in FieldTypes:
            if ft.value == field_type:
                return ft
        return None

In [30]:
# Graph Structure

class ModelNode:
    def __init__(self, name: str):
        self.name = name
        self.edges = []
        self.params = {'modules': set()} # This can be used to store any extra information about the model

    def add_edge(self, edge):
        # check if edge already exists
        for e in self.edges:
            if e.node1 == edge.node1 and e.node2 == edge.node2 and e.field == edge.field and e.field_type == edge.field_type:
                return
        self.edges.append(edge)

    def __str__(self):
        return self.name

ModelNode.all_nodes = {}

class RelationEdge:
    def __init__(self, node1: ModelNode, node2: ModelNode, field: str, field_type: str):
        self.node1 = node1
        self.node2 = node2
        self.field = field
        self.field_type = FieldTypes.get_field_type(field_type)

    def __str__(self):
        return self.node1.name + ' -> ' + self.node2.name + ' (' + self.field + ')'

In [38]:
def parse_line(lines: [str], idx, model_name, model_technical_name, module, is_model):

    line = lines[idx]

    # new python model class declaration
    if match := re.search(r'class\s+(\w+)\(models.((Model)|(TransientModel)|(AbstractModel))\):', line):
        model_name = match.group(1)
        is_model = True
        model_technical_name = ''

    # model technical name
    # https://regex101.com/r/DSHiTN/1
    elif match := re.search(r"""_name\s*=\s*('|\")([\w\.]*)('|\")""", line):
        model_technical_name = match.group(2)
        if ModelNode.all_nodes.get(model_technical_name) is None:
            ModelNode.all_nodes[model_technical_name] = ModelNode(model_technical_name)
        ModelNode.all_nodes[model_technical_name].params['modules'].add(module)


    elif match := re.search(r"""_inherit\s*=\s*('|\")([\w\.]*)('|\")""", line):
        if model_technical_name == '':
            model_technical_name = match.group(2)
        if ModelNode.all_nodes.get(model_technical_name) is None:
            ModelNode.all_nodes[model_technical_name] = ModelNode(model_technical_name)
        ModelNode.all_nodes[model_technical_name].params['modules'].add(module)


    # todo: Failing case: when inheriting a model, the inherited fields are not being detected

    # https://regex101.com/r/DSHiTN/1
    elif match := re.search(r"""_inherit\s*=\s*('|\")([\w\.]*)('|\")""", line):
        model_technical_name = match.group(2)

    # match field type
    for field_type in FIELD_TYPES:
        # regexr.com/86goi
        if match := re.search(r'([\w])+\s*=\s*fields\.((One2many\()|(Many2one\()|(Many2many\())', line):
            field_name = match.group(0).split('=')[0].strip()
            comodel_name = ''

            while ')' not in lines[idx] and idx < len(lines):
                idx += 1
                line += lines[idx]

            if 'comodel_name' in line and (match := re.search(r"""comodel_name\s*=\s*('|\")([\w\.]*)('|\")""", line)):
                comodel_name = match.group(2)

            elif match := re.search(r"""\(\s*('|\")([\w\.]*)('|\")""", line):
                comodel_name = match.group(2)

            if comodel_name:
                if ModelNode.all_nodes.get(comodel_name) is None:
                    ModelNode.all_nodes[comodel_name] = ModelNode(comodel_name)
                ModelNode.all_nodes[comodel_name].params['modules'].add(module)
                ModelNode.all_nodes[model_technical_name].add_edge(RelationEdge(ModelNode.all_nodes[model_technical_name], ModelNode.all_nodes[comodel_name], field_name, field_type))


    return idx, model_name, model_technical_name, is_model

In [39]:
# For each path, iterate through all python files and check if they contain any fields in them

for addon_path in addons_paths:
    for module in os.listdir(addon_path):
        if os.path.isdir(os.path.join(addon_path, module)):
            for folder in FOLDERS:
                folder_path = os.path.join(addon_path, module, folder)
                if os.path.exists(folder_path):
                    for file in os.listdir(folder_path):
                        if file.endswith('.py') and file != '__init__.py':
                            with open(os.path.join(folder_path, file), 'r') as f:
                                is_model = False
                                model_technical_name = ''
                                model_name = ''
                                lines = f.readlines()

                                for idx in range(len(lines)):
                                    idx, model_name, model_technical_name, is_model = parse_line(lines, idx, model_name, model_technical_name, module, is_model)

In [40]:
len(ModelNode.all_nodes)

1740

In [46]:
ModelNode.all_nodes['res.partner'].edges[20].field, ModelNode.all_nodes['res.partner'].edges[20].field_type

('picking_ids', <FieldTypes.One2many: 'One2many'>)

In [49]:
ModelNode.all_nodes['res.partner'].edges[20].node1.name, ModelNode.all_nodes['res.partner'].edges[20].node2.name

('res.partner', 'stock.picking')

In [79]:
def getLink(node1, node2):
    # get the link between two nodes by doing a breadth first search from node1 to node2
    visited = set()
    queue = [[node1]]
    while queue:
        path = queue.pop(0) # [node1, edge, node2]
        node = path[-1] # node2
        if node in visited:
            continue
        visited.add(node)
        for edge in node.edges:
            if edge.node2 == node2:
                return path + [edge, node2]
            queue.append(path + [edge, edge.node2])

    return None

In [81]:
path = getLink(ModelNode.all_nodes['sale.order'], ModelNode.all_nodes['account.move.line'])
[str(p) for p in path]

['sale.order',
 'sale.order -> res.partner (l10n_in_reseller_partner_id)',
 'res.partner',
 'res.partner -> account.move.line (unreconciled_aml_ids)',
 'account.move.line']

In [83]:
path = getLink(ModelNode.all_nodes['res.partner'], ModelNode.all_nodes['hr.employee'])
[str(p) for p in path]

['res.partner',
 'res.partner -> pos.order (pos_order_ids)',
 'pos.order',
 'pos.order -> hr.employee (employee_id)',
 'hr.employee']